# INTRODUÇÃO


_____________________________________



Nesta tarefa prática, estamos interessados em obter dados de quantidade de partos realizados em 2024 nos estados brasileiros, para responder os seguintes questionamentos:


1) Qual a proporção de partos cesáreos por UF?
2) Qual a quantidade e a proporção de partos cesáreos realizados por mês?
3) Qual a proporção de partos cesáreos realizados por faixa etária?
4) Qual a proporção de partos cesáreos por UF, realizados em beneficiárias jovens, com idades entre 15 e 19 anos?
5) Qual a proporção de partos cesáreos por UF, realizados em beneficiárias com idades entre 40 a 49 anos?
6) Qual o valor médio das despesas com partos por UF?





**Indicador de Partos Cesáreos**

O objetivo deste exercício é comparar a taxa de partos cesáreos realizados por beneficiárias de planos privados de saúde em 2024. Essa métrica é obtida dividindo-se o total de partos cesarianos pelo total de partos registrados dentro de cada unidade da federação, da seguinte forma:

 **Fórmula de Cálculo**

$$\text{Taxa de Cesarianas} = \frac{\text{Total de Partos Cesáreos}}{\text{Total de Partos (Vaginal + Múltiplo + Cesáreo)}} \times 100$$




**Filtros**

Selecionam-se exclusivamente os eventos ocorridos no ano de 2024, vinculados a planos de segmentação hospitalar com obstetrícia e beneficiárias com idades entre 15 e 49 anos. A identificação dos partos é realizada a partir dos códigos TUSS correspondentes aos procedimentos de parto vaginal, parto múltiplo, cesariana e cesariana com histerectomia.

* Unidade da Federação - UF
* Ano = 2024
* Planos com Segmentação Hospitalar com Obstetrícia
* Faixa etária de 15 a 49 anos

_____________________________________



O conjunto de dados reúne informações sobre procedimentos realizados em ambiente hospitalar por prestadores vinculados às operadoras de planos privados de saúde. As informações são disponibilizadas por unidade da federação, mês e ano de competência.



Os procedimentos hospitalares são disponibilizados em três conjuntos de dados distintos:

* Hospitalar Detalhado - (HOSP_DET)
* Hospitalar Consolidado - (HOSP_CONS)
* Hospitalar Modelo de Remuneração - (HOSP_REM)

Para este estudo, utilizam-se variáveis das bases HOSP_DET e HOSP_CONS. O dicionário completo das variáveis está disponível no Portal Brasileiro de Dados Abertos.



[

**Dicionário de Variáveis dos Procedimentos**](https://dados.gov.br/dados/conjuntos-dados/procedimentos-hospitalares-por-uf)



[

**Hospitalar Detalhado - HOSP_DET**](https://dados.gov.br/dados/conjuntos-dados/procedimentos-hospitalares-por-uf)

* **ID_EVENTO_ATENCAO_SAUDE** - Identificador único do evento
* **UF_PRESTADOR** - Unidade federativa do prestador executante
* **ANO_MES_EVENTO** - Ano e mês de ocorrência do evento
* **CD_PROCEDIMENTO** - Código do procedimento/ item assistencial (TUSS 19, 20, 22 ou 63)
* **CD_TABELA_REFERENCIA** - Código da tabela TUSS
* **QT_ITEM_EVENTO_INFORMADO** - Quantidade dos procedimentos/itens assistenciais individualizados ou do grupo de procedimentos (TUSS 63)
* **VL_ITEM_EVENTO_INFORMADO** - Valor informado dos procedimentos/itens assistenciais individualizados ou do grupo de procedimentos (TUSS 63)
* **IND_PACOTE** - Indica se o procedimento/item assistencial é parte de um pacote de procedimentos



[

**Hospitalar Consolidado - HOSP_CONS**](https://dados.gov.br/dados/conjuntos-dados/procedimentos-hospitalares-por-uf)

* **ID_EVENTO_ATENCAO_SAUDE** - Identificador único do evento
* **ID_PLANO** - Identificador único do plano
* **FAIXA_ETARIA** - Faixa etária na qual o beneficiário se enquadra em relação a sua idade
* **SEXO** - Identificação do sexo do beneficiário preenchida conforme a tabela TUSS 43 – Sexo

A TUSS define os termos padronizados utilizados para identificar eventos e itens assistenciais na saúde suplementar. Neste estudo, consideram-se os códigos referentes aos procedimentos de parto:





**Tabela TUSS 22 - Procedimentos e Eventos em Saúde**

* 31309127	- Parto via vaginal
* 31309135	- Parto múltiplo
* 31309054	- Cesariana
* 31309208	- Cesariana com histerectomia


# CONFIGURAÇÕES

In [ ]:
######################################
# Instalando e carregando bibliotecas
######################################

# Instala pacotes se necessário
# install.packages(c('tidyverse', 'readxl', 'writexl', 'RCurl', 'ggplot2', 'plotly', 'stringr'))

# Carrega as bibliotecas necessárias
library(tidyverse)      # Para manipulação de dados
library(readxl)         # Para ler Excel
library(writexl)        # Para salvar em Excel
library(RCurl)          # Para download via URL
library(ggplot2)        # Para visualização
library(plotly)         # Para gráficos interativos
library(stringr)        # Para manipulação de strings

In [ ]:
# Conectando ao Google Drive

library(googledrive)

# Autentica no Google Drive
drive_auth()

# Define o diretório de trabalho
setwd('/content/drive/MyDrive/OFICINA_DESID')

In [ ]:
###############################
### CONFIGURAÇÕES
###############################

# Define opções de exibição
options(max.print = 10000)
options(digits = 4)

In [ ]:
###############################
### CONFIGURAÇÕES DE CORES
###############################

# Cores definidas
laranja <- "#F47521"
verde_claro <- "#6D983F"
azul <- "#006E89"
amarelo <- "#d6bf16"
marrom <- "#683900"
azul_claro <- "#98afcc"
verde <- "#195214"

In [ ]:
###############################
# PARÂMETROS PARA AS EXTRAÇÕES
###############################

# Lista de UFs
ufs <- c(
  "AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MG", "MS", "MT",
  "PA", "PB", "PE", "PI", "PR", "RJ", "RN", "RO", "RR", "RS", "SC", "SE", "SP", "TO"
)

cat(sprintf('Tamanho da lista de UFs: %d\n', length(ufs)))

# Lista de códigos TUSS de interesse
tuss <- c('31309127', '31309135', '31309054', '31309208')

In [ ]:
stop("Pausa para configuração")

# BASE DE DADOS

### Hospitalar Detalhada

In [ ]:
##########################################################################
# BAIXANDO DADOS DE PROCEDIMENTOS HOSPITALARES POR UF - BASES DETALHADAS
##########################################################################

####################################################################################################
# Configuração

# Colunas desejadas
colunas <- c(
  'ID_EVENTO_ATENCAO_SAUDE',
  'ANO_MES_EVENTO',
  'UF_PRESTADOR',
  'CD_PROCEDIMENTO',
  'CD_TABELA_REFERENCIA',
  'QT_ITEM_EVENTO_INFORMADO',
  'VL_ITEM_EVENTO_INFORMADO',
  'IND_PACOTE'
)

####################################################################################################
# Extração

# Lista para armazenar todos os DataFrames
dfs <- list()

# Loop pelas UFs e meses
for (uf in ufs) {
  cat(sprintf("\\n########## Iniciando UF: %s ##########\\n", uf))
  
  for (mes in 1:12) {
    zip_url <- sprintf(
      "https://dadosabertos.ans.gov.br/FTP/PDA/TISS/HOSPITALAR/2024/%s/%s_2024%02d_HOSP_DET.zip",
      uf, uf, mes
    )
    cat(sprintf("Baixando: %s\\n", zip_url))
    
    tryCatch({
      # Download do arquivo ZIP
      temp_zip <- tempfile(fileext = ".zip")
      download.file(zip_url, temp_zip, mode = "wb", timeout = 60, quiet = TRUE)
      
      # Descompacta e lê o arquivo
      temp_dir <- tempdir()
      unzip(temp_zip, exdir = temp_dir, overwrite = TRUE)
      
      # Encontra o nome do arquivo CSV
      csv_files <- list.files(temp_dir, pattern = "\\.csv$", full.names = TRUE)
      
      if (length(csv_files) > 0) {
        df_temp <- read_delim(
          csv_files[1],
          delim = ";",
          locale = locale(encoding = "CP1252"),
          col_types = cols(.default = col_character())
        ) %>%
          select(all_of(colunas)) %>%
          filter(CD_PROCEDIMENTO %in% tuss)
        
        dfs[[length(dfs) + 1]] <- df_temp
        
        cat(sprintf("%s %02d: %s linhas carregadas\\n", uf, mes, nrow(df_temp)))
      }
      
      # Limpa arquivos temporários
      unlink(temp_zip)
      unlink(csv_files)
      
    }, error = function(e) {
      cat(sprintf("\\n Falha ao processar %s %02d: %s\\n", uf, mes, conditionMessage(e)))
    })
  }
}

# Junta todos os DataFrames
if (length(dfs) > 0) {
  df_det <- bind_rows(dfs)
  cat(sprintf("\\n Base detalhada criada com %d linhas e %d colunas.\\n", nrow(df_det), ncol(df_det)))
} else {
  cat("\\n Nenhum arquivo foi carregado com sucesso.\\n")
}

####################################################################################################
# Checagem

head(df_det)

### Hospitalar Consolidada

In [ ]:
##########################################################################
# INFORMAÇÕES PARA EXTRAÇÃO DA BASE CONSOLIDADA
##########################################################################

# Objetivo: abrir informações consolidadas somente nos eventos que apareceram na base detalhada.

# Gera a lista de valores únicos
ids_eventos <- unique(df_det$ID_EVENTO_ATENCAO_SAUDE)

# Mostra quantos valores únicos existem
cat(sprintf("Total de valores únicos: %d\\n", length(ids_eventos)))

# Exibe os 10 primeiros valores
head(ids_eventos, 10)

In [ ]:
##############################################################################
# BAIXANDO DADOS DE PROCEDIMENTOS HOSPITALARES POR UF - BASES CONSOLIDADAS
##############################################################################

####################################################################################################
# Configuração

# Colunas desejadas
colunas <- c('ID_EVENTO_ATENCAO_SAUDE', 'ID_PLANO', 'FAIXA_ETARIA', 'SEXO')

####################################################################################################
# Extração

# Lista para armazenar todos os DataFrames
dfs <- list()

for (uf in ufs) {
  cat(sprintf("\\n########## Iniciando UF: %s ##########\\n", uf))
  
  for (mes in 1:12) {
    zip_url <- sprintf(
      "https://dadosabertos.ans.gov.br/FTP/PDA/TISS/HOSPITALAR/2024/%s/%s_2024%02d_HOSP_CONS.zip",
      uf, uf, mes
    )
    cat(sprintf("Baixando: %s\\n", zip_url))
    
    tryCatch({
      # Download do arquivo ZIP
      temp_zip <- tempfile(fileext = ".zip")
      download.file(zip_url, temp_zip, mode = "wb", timeout = 60, quiet = TRUE)
      
      # Descompacta e lê o arquivo
      temp_dir <- tempdir()
      unzip(temp_zip, exdir = temp_dir, overwrite = TRUE)
      
      # Encontra o nome do arquivo CSV
      csv_files <- list.files(temp_dir, pattern = "\\.csv$", full.names = TRUE)
      
      if (length(csv_files) > 0) {
        df_temp <- read_delim(
          csv_files[1],
          delim = ";",
          locale = locale(encoding = "latin1"),
          col_types = cols(.default = col_character())
        ) %>%
          select(all_of(colunas)) %>%
          filter(ID_EVENTO_ATENCAO_SAUDE %in% ids_eventos)
        
        dfs[[length(dfs) + 1]] <- df_temp
        
        cat(sprintf("%s %02d: %d linhas carregadas\\n", uf, mes, nrow(df_temp)))
      }
      
      # Limpa arquivos temporários
      unlink(temp_zip)
      unlink(csv_files)
      
    }, error = function(e) {
      cat(sprintf("\\n Falha ao processar %s %02d: %s\\n", uf, mes, conditionMessage(e)))
    })
  }
}

# Junta todos os DataFrames
if (length(dfs) > 0) {
  df_cons <- bind_rows(dfs)
  cat(sprintf("\\n Base consolidada criada com %d linhas e %d colunas.\\n", nrow(df_cons), ncol(df_cons)))
} else {
  cat("\\n Nenhum arquivo foi carregado com sucesso.\\n")
}

####################################################################################################
# Checagem

head(df_cons)

### Base de Planos

In [ ]:
###########################
# ABRINDO BASE DE PLANOS
###########################

# Link do arquivo ZIP
url <- "https://dadosabertos.ans.gov.br/FTP/PDA/TISS/DADOS_DE_PLANOS/PLANOS.zip"

# Download do arquivo ZIP em memória
temp_zip <- tempfile(fileext = ".zip")
download.file(url, temp_zip, mode = "wb", quiet = TRUE)

# Descompacta e lê o arquivo
temp_dir <- tempdir()
unzip(temp_zip, exdir = temp_dir, overwrite = TRUE)

# Encontra o nome do arquivo CSV
csv_files <- list.files(temp_dir, pattern = "\\.csv$", full.names = TRUE)

# Colunas desejadas
colunas <- c('ID_PLANO', 'CD_SEGMENTACAO', 'TIPO_SEGMENTACAO')

# Lê o arquivo
df_plano <- read_delim(
  csv_files[1],
  delim = ";",
  locale = locale(encoding = "latin1"),
  col_types = cols(.default = col_character())
) %>%
  select(all_of(colunas))

# Limpa arquivos temporários
unlink(temp_zip)
unlink(csv_files)

####################################################################################################
# Checagem

head(df_plano)

### Juntando Bases

In [ ]:
##############################
# CHECAGEM NAS BASES
###############################

# Verificando dimensões das bases
cat(sprintf("\\nShape Base DET: %d linhas, %d colunas\\n", nrow(df_det), ncol(df_det)))
cat(sprintf("\\nShape Base CONS: %d linhas, %d colunas\\n", nrow(df_cons), ncol(df_cons)))
cat(sprintf("\\nShape Base PLANO: %d linhas, %d colunas\\n", nrow(df_plano), ncol(df_plano)))

cat(sprintf("\\nColunas df_det: %s\\n", paste(names(df_det), collapse = ", ")))
cat(sprintf("\\nColunas df_cons: %s\\n", paste(names(df_cons), collapse = ", ")))
cat(sprintf("\\nColunas df_plano: %s\\n", paste(names(df_plano), collapse = ", ")))

In [ ]:
##############################################################################
# JUNTANDO BASE CONSOLIDADA COM BASE DE PLANO
##############################################################################

# Faz o merge pela ID_PLANO
df_merged <- df_cons %>%
  left_join(
    df_plano %>% select(ID_PLANO, CD_SEGMENTACAO, TIPO_SEGMENTACAO),
    by = "ID_PLANO"
  )

# Verifica se houve perda de informação
missing_segmentacao <- sum(is.na(df_merged$CD_SEGMENTACAO))
missing_tipo <- sum(is.na(df_merged$TIPO_SEGMENTACAO))

cat(sprintf("\\nShape df_cons: %d linhas, %d colunas\\n", nrow(df_cons), ncol(df_cons)))
cat(sprintf("\\nShape df_merged: %d linhas, %d colunas\\n", nrow(df_merged), ncol(df_merged)))

cat(sprintf("Linhas com CD_SEGMENTACAO ausente: %d\\n", missing_segmentacao))
cat(sprintf("Linhas com TIPO_SEGMENTACAO ausente: %d\\n", missing_tipo))

In [ ]:
##############################################################################
# JUNTANDO BASE MERGIDA (CONSOLIDADA + PLANO) E BASE DETALHADA
##############################################################################

# Faz o merge (left join) para manter todas as linhas do df_det
df_final <- df_det %>%
  left_join(
    df_merged %>% select(ID_EVENTO_ATENCAO_SAUDE, FAIXA_ETARIA, SEXO, CD_SEGMENTACAO, TIPO_SEGMENTACAO),
    by = "ID_EVENTO_ATENCAO_SAUDE"
  )

# Verifica se há valores ausentes nas colunas trazidas
missing_cd <- sum(is.na(df_final$CD_SEGMENTACAO))
missing_tipo <- sum(is.na(df_final$TIPO_SEGMENTACAO))
missing_sexo <- sum(is.na(df_final$SEXO))
missing_faixa <- sum(is.na(df_final$FAIXA_ETARIA))

cat(sprintf("\\nShape df_det: %d linhas, %d colunas\\n", nrow(df_det), ncol(df_det)))
cat(sprintf("\\nShape df_merged: %d linhas, %d colunas\\n", nrow(df_merged), ncol(df_merged)))
cat(sprintf("\\nShape df_final: %d linhas, %d colunas\\n", nrow(df_final), ncol(df_final)))

cat(sprintf("Linhas com CD_SEGMENTACAO ausente: %d\\n", missing_cd))
cat(sprintf("Linhas com TIPO_SEGMENTACAO ausente: %d\\n", missing_tipo))
cat(sprintf("Linhas com SEXO ausente: %d\\n", missing_sexo))
cat(sprintf("Linhas com FAIXA_ETARIA ausente: %d\\n", missing_faixa))

In [ ]:
##############################
# CHECAGEM NA BASE
###############################

cat(sprintf("\\nShape: %d linhas, %d colunas\\n", nrow(df_final), ncol(df_final)))
cat(sprintf("\\nColunas: %s\\n", paste(names(df_final), collapse = ", ")))

head(df_final)

### Salvando Base Final

In [ ]:
#########################
# Salvando base final
#########################

# Caminho de saída
caminho_saida <- "base_hospitalar_2024.xlsx"

# Salva em Excel
write_xlsx(df_final, caminho_saida)

# LIMPEZA DE DADOS

In [ ]:
##############################
# CHECAGEM NA BASE
###############################

# Caminho do arquivo
file <- "base_hospitalar_2024.xlsx"

# Lendo o arquivo Excel
df <- read_excel(file)

# Verificando as primeiras linhas
cat(sprintf("\\nShape: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

head(df)

In [ ]:
# Informações sobre a estrutura
str(df)
summary(df)

## Checagem

In [ ]:
########################################
# CHECAGEM DE INFORMAÇÕES DOS MESES
########################################

cat(sprintf("\\nQt. Meses: %d\\n", n_distinct(df$ANO_MES_EVENTO)))

# Tabela de frequência com percentual
df %>%
  group_by(ANO_MES_EVENTO) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

In [ ]:
########################################
# CHECAGEM DE INFORMAÇÕES DAS UFS
########################################

cat(sprintf("\\nQt. UFs: %d\\n", n_distinct(df$UF_PRESTADOR)))

# Tabela de frequência com percentual
df %>%
  group_by(UF_PRESTADOR) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  arrange(desc(n)) %>%
  print()

In [ ]:
################################################
# ANÁLISE DO CONTEÚDO DAS VARIÁVEIS
################################################

# Lista de colunas categóricas
column_list <- c('CD_PROCEDIMENTO', 'CD_TABELA_REFERENCIA', 'IND_PACOTE',
                  'FAIXA_ETARIA', 'SEXO', 'CD_SEGMENTACAO', 'TIPO_SEGMENTACAO')

# Cria plots para cada variável
plots <- list()

for (col in column_list) {
  p <- df %>%
    group_by(!!sym(col)) %>%
    summarise(n = n()) %>%
    ggplot(aes(x = reorder(!!sym(col), -n), y = n, fill = azul)) +
    geom_col() +
    geom_text(aes(label = n), vjust = -0.5) +
    theme_minimal() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
    labs(title = col, x = col, y = "Frequência") +
    theme(legend.position = "none")
  
  plots[[col]] <- p
}

# Exibe os plots (em Colab, use gridExtra::grid.arrange ou plotly)
for (plot in plots) {
  print(plot)
}

In [ ]:
# Resumo da estrutura dos dados
glimpse(df)

## Formatação

### Tratando Variáveis Numéricas

In [ ]:
################################################
# CONVERSÃO DE TIPO PARA COLUNAS NUMÉRICAS
################################################

cols_num <- c('QT_ITEM_EVENTO_INFORMADO', 'VL_ITEM_EVENTO_INFORMADO')

# Converte para numérico
df <- df %>%
  mutate(across(
    all_of(cols_num),
    ~ as.numeric(str_replace(., ",", "."))
  ))

# Verifica tipos de dados
df %>%
  select(all_of(cols_num)) %>%
  sapply(class) %>%
  print()

# Estatísticas descritivas
df %>%
  select(all_of(cols_num)) %>%
  summary()

### Criando Variáveis

In [ ]:
##################################################################################
# CRIANDO VARIÁVEL INDICADORA DE OUTLIER PELO MÉTODO DO INTERVALO INTERQUARTIL
##################################################################################

# Cálculo do IQR
Q1 <- quantile(df$VL_ITEM_EVENTO_INFORMADO, 0.25, na.rm = TRUE)
Q3 <- quantile(df$VL_ITEM_EVENTO_INFORMADO, 0.75, na.rm = TRUE)
IQR_val <- Q3 - Q1

##########################
# Limites
limite_inferior <- Q1 - 1.5 * IQR_val
limite_superior <- Q3 + 1.5 * IQR_val

##########################
# Criação da variável OUTLIER
df <- df %>%
  mutate(OUTLIER = if_else(
    VL_ITEM_EVENTO_INFORMADO < limite_inferior | VL_ITEM_EVENTO_INFORMADO > limite_superior,
    1, 0
  ))

##########################
# Exibir limites
cat(sprintf("Limite inferior: %f\\n", limite_inferior))
cat(sprintf("Limite superior: %f\\n", limite_superior))

##########################
# Visualizar primeiros resultados
df %>%
  select(VL_ITEM_EVENTO_INFORMADO, OUTLIER) %>%
  head()

In [ ]:
###################################################
# CRIANDO VARIÁVEL COM DESCRIÇÃO DO PROCEDIMENTO
###################################################

# Dicionário de mapeamento
map_procedimento <- c(
  '31309127' = 'Parto via vaginal',
  '31309135' = 'Parto múltiplo',
  '31309054' = 'Cesariana',
  '31309208' = 'Cesariana com histerectomia'
)

# Criando nova variável
df <- df %>%
  mutate(
    CD_PROCEDIMENTO = as.character(CD_PROCEDIMENTO),
    PROCEDIMENTO = recode(CD_PROCEDIMENTO, !!!map_procedimento)
  )

# Checagem de informações de Procedimentos
cat(sprintf("\\nQuantidade:\\n"))
df %>%
  group_by(PROCEDIMENTO) %>%
  summarise(n = n()) %>%
  print()

cat(sprintf("\\nPercentual:\\n"))
df %>%
  group_by(PROCEDIMENTO) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

In [ ]:
############################################################
# CRIANDO A VARIÁVEL MÊS COM A DESCRIÇÃO
############################################################

# Extrai o número do mês
df <- df %>%
  mutate(
    MES_NUM = as.numeric(str_extract(ANO_MES_EVENTO, "(?<=-)\\d{2}$")),
    MES = recode(
      MES_NUM,
      `1` = 'JAN', `2` = 'FEV', `3` = 'MAR', `4` = 'ABR',
      `5` = 'MAI', `6` = 'JUN', `7` = 'JUL', `8` = 'AGO',
      `9` = 'SET', `10` = 'OUT', `11` = 'NOV', `12` = 'DEZ'
    )
  )

In [ ]:
###########################################################
# ANALISANDO DADOS DE SEGMENTAÇÃO
###########################################################

df_seg <- df %>%
  group_by(CD_SEGMENTACAO, TIPO_SEGMENTACAO) %>%
  summarise(QT_REGISTROS = n(), .groups = 'drop') %>%
  mutate(PCT_REGISTROS = (QT_REGISTROS / sum(QT_REGISTROS)) * 100) %>%
  arrange(desc(QT_REGISTROS))

print(df_seg)

In [ ]:
###########################################################
# CRIANDO VARIÁVEL PARA INDICAR PLANO COM OBSTETRÍCIA
###########################################################

planos_sem_obstetricia <- c(4, 7, 1, 8, 14)
planos_obstetricia <- c(2, 5, 6, 9, 10, 12, 13, 15)

df <- df %>%
  mutate(
    CD_SEGMENTACAO = as.numeric(CD_SEGMENTACAO),
    OBSTETRICIA = if_else(CD_SEGMENTACAO %in% planos_obstetricia, 1, 0)
  )

cat(sprintf("\\nQuantidade:\\n"))
df %>%
  group_by(OBSTETRICIA) %>%
  summarise(n = n()) %>%
  print()

cat(sprintf("\\nPercentual:\\n"))
df %>%
  group_by(OBSTETRICIA) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

In [ ]:
##########################
# CHECAGEM DO RESULTADO
##########################

# Quantidade sem obstetrícia
qt_sem <- df %>%
  filter(CD_SEGMENTACAO %in% planos_sem_obstetricia) %>%
  nrow()

# Quantidade com obstetrícia
qt_com <- df %>%
  filter(CD_SEGMENTACAO %in% planos_obstetricia) %>%
  nrow()

cat(sprintf("Quantidade sem obstetrícia: %d\\n", qt_sem))
cat(sprintf("Quantidade com obstetrícia: %d\\n", qt_com))

## Aplicando Filtros

### Faixa Etária

In [ ]:
############################################
# CHECAGEM DE INFORMAÇÕES POR FAIXA ETÁRIA
############################################

cat(sprintf("\\nOpções de FAIXA_ETARIA:\\n"))
print(unique(df$FAIXA_ETARIA))

cat(sprintf("\\nQuantidade por faixa etária:\\n"))
df %>%
  group_by(FAIXA_ETARIA) %>%
  summarise(n = n()) %>%
  print()

cat(sprintf("\\nPercentual por faixa etária:\\n"))
df %>%
  group_by(FAIXA_ETARIA) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

In [ ]:
############################################
# FILTRANDO FAIXAS ETÁRIAS DE INTERESSE
############################################

cat(sprintf("\\nShape antes: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

df <- df %>%
  filter(FAIXA_ETARIA %in% c('15 a 19', '20 a 29', '30 a 39', '40 a 49'))

cat(sprintf("\\nPercentual por faixa etária (após filtro):\\n"))
df %>%
  group_by(FAIXA_ETARIA) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

cat(sprintf("\\nShape depois: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

In [ ]:
#########################
# CHECAGEM DO RESULTADO
#########################

# Define a ordem desejada
ordem_faixas <- c('15 a 19', '20 a 29', '30 a 39', '40 a 49')

# Converte para fator ordenado
df <- df %>%
  mutate(FAIXA_ETARIA = factor(FAIXA_ETARIA, levels = ordem_faixas, ordered = TRUE))

# Plot
p <- df %>%
  group_by(FAIXA_ETARIA) %>%
  summarise(n = n()) %>%
  ggplot(aes(x = FAIXA_ETARIA, y = n, fill = verde)) +
  geom_col(alpha = 0.7, color = "black") +
  geom_text(aes(label = n), vjust = -0.5) +
  theme_minimal() +
  labs(title = "FAIXA_ETARIA", x = "FAIXA_ETARIA", y = "Frequência") +
  theme(legend.position = "none", axis.text.x = element_text(angle = 45, hjust = 1))

print(p)

### Sexo

In [ ]:
############################################
# CHECAGEM DE INFORMAÇÕES POR SEXO
############################################

cat(sprintf("\\nQuantidade por sexo:\\n"))
df %>%
  group_by(SEXO) %>%
  summarise(n = n()) %>%
  print()

cat(sprintf("\\nPercentual por sexo:\\n"))
df %>%
  group_by(SEXO) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

In [ ]:
############################################
# LIMPANDO DADOS DE SEXO MASCULINO
############################################

cat(sprintf("\\nShape antes: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

df <- df %>%
  mutate(SEXO = if_else(SEXO == "Masculino", "Feminino", SEXO))

cat(sprintf("\\nShape depois: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

cat(sprintf("\\nPercentual por sexo (após limpeza):\\n"))
df %>%
  group_by(SEXO) %>%
  summarise(n = n(), pct = n / nrow(df)) %>%
  print()

### Segmentação

In [ ]:
############################################
# RETIRANDO PLANOS SEM OBSTETRÍCIA
############################################

cat(sprintf("\\nShape antes: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

df <- df %>%
  filter(OBSTETRICIA == 1)

cat(sprintf("\\nShape depois: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

In [ ]:
############################################
# CHECAGEM DE RESULTADOS
############################################

# Plot
p <- df %>%
  group_by(TIPO_SEGMENTACAO) %>%
  summarise(n = n()) %>%
  ggplot(aes(x = reorder(TIPO_SEGMENTACAO, -n), y = n, fill = verde)) +
  geom_col(alpha = 0.7, color = "black") +
  geom_text(aes(label = n), vjust = -0.5) +
  theme_minimal() +
  labs(title = "TIPO_SEGMENTACAO", x = "TIPO_SEGMENTACAO", y = "Frequência") +
  theme(legend.position = "none", axis.text.x = element_text(angle = 45, hjust = 1))

print(p)

### Códigos TUSS

In [ ]:
############################################
# ANALISANDO DADOS DE PROCEDIMENTOS
############################################

# Plot
p <- df %>%
  group_by(PROCEDIMENTO) %>%
  summarise(n = n()) %>%
  ggplot(aes(x = reorder(PROCEDIMENTO, -n), y = n, fill = verde)) +
  geom_col(alpha = 0.7, color = "black") +
  geom_text(aes(label = n), vjust = -0.5) +
  theme_minimal() +
  labs(title = "PROCEDIMENTO", x = "PROCEDIMENTO", y = "Frequência") +
  theme(legend.position = "none", axis.text.x = element_text(angle = 45, hjust = 1))

print(p)

In [ ]:
##########################################################
# SUBSTITUIR CESARIANA COM HISTERECTOMIA PARA CESARIANA
##########################################################

df <- df %>%
  mutate(
    CD_PROCEDIMENTO = as.character(CD_PROCEDIMENTO),
    PROCEDIMENTO = if_else(PROCEDIMENTO == "Cesariana com histerectomia", "Cesariana", PROCEDIMENTO),
    CD_PROCEDIMENTO = if_else(CD_PROCEDIMENTO == "31309208", "31309054", CD_PROCEDIMENTO)
  )

In [ ]:
############################################
# CHECAGEM DE RESULTADOS
############################################

# Plot
p <- df %>%
  group_by(PROCEDIMENTO) %>%
  summarise(n = n()) %>%
  ggplot(aes(x = reorder(PROCEDIMENTO, -n), y = n, fill = verde)) +
  geom_col(alpha = 0.7, color = "black") +
  geom_text(aes(label = n), vjust = -0.5) +
  theme_minimal() +
  labs(title = "PROCEDIMENTO", x = "PROCEDIMENTO", y = "Frequência") +
  theme(legend.position = "none", axis.text.x = element_text(angle = 45, hjust = 1))

print(p)

## Salvando Base Tratada

In [ ]:
##############################
# CHECAGEM NA BASE
###############################

cat(sprintf("\\nShape: %d linhas, %d colunas\\n", nrow(df), ncol(df)))
cat(sprintf("\\nColunas: %s\\n", paste(names(df), collapse = ", ")))

head(df)

In [ ]:
####################################
# DELETAR COLUNAS DESNECESSÁRIAS
####################################

cat(sprintf("\\nShape antes: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

colunas_para_remover <- c('SEXO', 'CD_SEGMENTACAO', 'TIPO_SEGMENTACAO', 'OBSTETRICIA', 'MES_NUM')

df <- df %>%
  select(-all_of(colunas_para_remover))

cat(sprintf("\\nShape depois: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

In [ ]:
# Resumo da estrutura
glimpse(df)

In [ ]:
#########################
# SALVANDO BASE TRATADA
#########################

# Caminho de saída
caminho_saida <- "base_hospitalar_tratada_2024.xlsx"

# Salva em Excel
write_xlsx(df, caminho_saida)

cat(sprintf("\\nArquivo salvo com sucesso em: %s\\n", caminho_saida))

# ANÁLISE DESCRITIVA

In [ ]:
##############################
# CHECAGEM NA BASE
###############################

# Caminho do arquivo
file <- "base_hospitalar_tratada_2024.xlsx"

# Lendo o arquivo Excel
df <- read_excel(file)

# Verificando as primeiras linhas
cat(sprintf("\\nShape: %d linhas, %d colunas\\n", nrow(df), ncol(df)))

head(df)

In [ ]:
# Resumo da estrutura
glimpse(df)

## Valores ausentes

In [ ]:
##############################
# ESTUDANDO VALORES AUSENTES
###############################

# Calcula quantidade e percentual de valores ausentes
tabela_null <- data.frame(
  VARIAVEL = names(df),
  QT_REGISTROS = colSums(is.na(df)),
  row.names = NULL
) %>%
  filter(QT_REGISTROS > 0) %>%
  mutate(
    PCT_REGISTROS = (QT_REGISTROS / nrow(df)) * 100
  ) %>%
  arrange(desc(QT_REGISTROS))

print(tabela_null)

In [ ]:
##############################
# VISUALIZANDO VALORES AUSENTES
###############################

# Calcula padrão de valores ausentes por coluna
missing_pattern <- df %>%
  summarise(across(everything(), ~ sum(is.na(.))))

print(missing_pattern)

In [ ]:
####################################################
# ESTUDANDO VALORES AUSENTES EM DADOS DE PACOTES
#####################################################

df_pacote1 <- df %>%
  filter(IND_PACOTE == 1)

# Quantidade de valores ausentes por variável
df_pacote1 %>%
  summarise(across(everything(), ~ sum(is.na(.)))) %>%
  print()

In [ ]:
############################################################
# ESTUDANDO VALORES AUSENTES EM DADOS QUE NÃO SÃO PACOTES
#############################################################

df_pacote0 <- df %>%
  filter(IND_PACOTE == 0)

# Quantidade de valores ausentes por variável
df_pacote0 %>%
  summarise(across(everything(), ~ sum(is.na(.)))) %>%
  print()

## Pacotes

In [ ]:
##############################
# DADOS DE PACOTES
###############################

# Quantidade total de registros
total <- nrow(df)

# Agrupa e calcula estatísticas
tabela_pacotes <- df %>%
  group_by(IND_PACOTE) %>%
  summarise(
    QT = n(),
    MEDIA_QT_ITEM = mean(QT_ITEM_EVENTO_INFORMADO, na.rm = TRUE),
    MEDIA_VL_ITEM = mean(VL_ITEM_EVENTO_INFORMADO, na.rm = TRUE),
    .groups = 'drop'
  ) %>%
  mutate(
    SITUACAO = if_else(IND_PACOTE == 0, "IND_PACOTE = 0", "IND_PACOTE = 1"),
    PCT_REGISTROS = (QT / total) * 100
  ) %>%
  select(SITUACAO, QT, PCT_REGISTROS, MEDIA_QT_ITEM, MEDIA_VL_ITEM)

cat(sprintf("\\nQt. total de registros: %d\\n", total))
print(tabela_pacotes)

## Valor Informado

In [ ]:
# Estatísticas descritivas
df$VL_ITEM_EVENTO_INFORMADO %>%
  summary() %>%
  print()

In [ ]:
######################################
# ANALISANDO VL_ITEM_EVENTO_INFORMADO
######################################

# Lista de percentis
percentis <- c(0.90, 0.95, 0.97, 0.99, 0.995, 0.999)
nomes_percentis <- c('P90', 'P95', 'P97', 'P99', 'P99_5', 'P99_9')

resultados <- list()
col <- 'VL_ITEM_EVENTO_INFORMADO'
total_registros <- nrow(df)

for (i in seq_along(percentis)) {
  valor_percentil <- quantile(df[[col]], percentis[i], na.rm = TRUE)
  
  qt_registros <- sum(df[[col]] > valor_percentil, na.rm = TRUE)
  pct_registros <- qt_registros / total_registros
  media_acima <- mean(df[df[[col]] > valor_percentil, ][[col]], na.rm = TRUE)
  
  resultados[[i]] <- list(
    METRICA = nomes_percentis[i],
    VL_MEDIO = media_acima,
    QT_REGISTROS = qt_registros,
    PCT_REGISTROS = pct_registros
  )
}

# Cria data frame a partir dos resultados
tabela_metricas <- do.call(rbind, lapply(resultados, as.data.frame))

print(tabela_metricas)

In [ ]:
##############################################################
# ANALISANDO VL_ITEM_EVENTO_INFORMADO SEM OUTLIERS
##############################################################

df_sem_outlier <- df %>%
  filter(OUTLIER == 0)

df_sem_outlier$VL_ITEM_EVENTO_INFORMADO %>%
  summary() %>%
  print()

In [ ]:
# Estatísticas descritivas com outliers
df$VL_ITEM_EVENTO_INFORMADO %>%
  summary() %>%
  print()

## QT Informada

In [ ]:
######################################
# ANALISANDO QT_ITEM_EVENTO_INFORMADO
######################################

# Quantidade de registros por valor de QT
df %>%
  group_by(QT_ITEM_EVENTO_INFORMADO) %>%
  summarise(n = n()) %>%
  arrange(desc(n)) %>%
  print()

In [ ]:
######################################
# FILTRANDO REGISTROS COM QT > 3
######################################

# Define as colunas de interesse
colunas_interesse <- c(
  'ID_EVENTO_ATENCAO_SAUDE', 'PROCEDIMENTO',
  'CD_TABELA_REFERENCIA', 'VL_ITEM_EVENTO_INFORMADO',
  'QT_ITEM_EVENTO_INFORMADO', 'IND_PACOTE'
)

# Filtra registros com mais de 3 itens
df_novo <- df %>%
  filter(QT_ITEM_EVENTO_INFORMADO > 3) %>%
  select(all_of(colunas_interesse))

print(df_novo)

## Múltiplos procedimentos por evento

In [ ]:
######################################
# ANALISANDO PROCEDIMENTOS POR EVENTO
######################################

# Descobre IDs que possuem mais de um CD_PROCEDIMENTO
ids_multiplos <- df %>%
  group_by(ID_EVENTO_ATENCAO_SAUDE) %>%
  summarise(n_procedimentos = n_distinct(CD_PROCEDIMENTO), .groups = 'drop') %>%
  filter(n_procedimentos > 1) %>%
  pull(ID_EVENTO_ATENCAO_SAUDE)

# Filtra df mantendo apenas esses IDs
df_multiplos <- df %>%
  filter(ID_EVENTO_ATENCAO_SAUDE %in% ids_multiplos)

cat(sprintf("Qt. eventos com múltiplos códigos de procedimentos: %d\\n", n_distinct(df_multiplos$ID_EVENTO_ATENCAO_SAUDE)))

df_multiplos %>%
  select(ID_EVENTO_ATENCAO_SAUDE, PROCEDIMENTO, QT_ITEM_EVENTO_INFORMADO, VL_ITEM_EVENTO_INFORMADO) %>%
  print()

In [ ]:
#########################
# SALVANDO DADOS MÚLTIPLOS
#########################

# Pode salvar se necessário:
# write_xlsx(df_multiplos, "df_multiplos.xlsx")

# CÁLCULO DO INDICADOR

## Partos Cesáreos por UF

In [ ]:
############################################################
# CÁLCULO DO INDICADOR DE PARTOS CESÁREAS POR UF
############################################################

# Somando a QT de partos por tipo e UF
df_agg <- df %>%
  group_by(PROCEDIMENTO, UF_PRESTADOR) %>%
  summarise(QT_ITEM_EVENTO_INFORMADO = sum(QT_ITEM_EVENTO_INFORMADO), .groups = 'drop')

########################################
# ORGANIZANDO A BASE

df_pivot <- df_agg %>%
  pivot_wider(
    id_cols = UF_PRESTADOR,
    names_from = PROCEDIMENTO,
    values_from = QT_ITEM_EVENTO_INFORMADO,
    values_fill = 0
  ) %>%
  rename(
    UF = UF_PRESTADOR,
    QT_Cesariana = `Cesariana`,
    QT_Parto_multiplo = `Parto múltiplo`,
    QT_Parto_via_vaginal = `Parto via vaginal`
  )

########################################
# CALCULANDO TOTAL DE PARTOS
df_pivot <- df_pivot %>%
  mutate(
    QT_TOTAL_PARTOS = QT_Cesariana + QT_Parto_multiplo + QT_Parto_via_vaginal,
    TAXA_CESAREA = (QT_Cesariana / QT_TOTAL_PARTOS) * 100
  )

print(df_pivot)

### Figura 1

In [ ]:
##############################################################
### TAXA DE PARTOS CESÁREOS POR UF
##############################################################

# Ordenando os dados por 'TAXA_CESAREA' de forma decrescente
df_plot <- df_pivot %>%
  arrange(desc(TAXA_CESAREA))

# Criando o gráfico de barras
p <- ggplot(df_plot, aes(x = reorder(UF, TAXA_CESAREA), y = TAXA_CESAREA, fill = TAXA_CESAREA)) +
  geom_col() +
  scale_fill_gradient(low = azul_claro, high = azul) +
  geom_text(aes(label = round(TAXA_CESAREA, 0)), vjust = -0.5, size = 3) +
  coord_flip() +
  theme_minimal() +
  labs(
    title = "Taxa de Partos Cesáreos por UF em 2024",
    x = "UF",
    y = "Taxa de Cesarianas (%)",
    fill = "Taxa (%)"
  ) +
  theme(legend.position = "right")

print(p)

# Versão interativa com plotly
p_interactive <- plot_ly(
  data = df_plot,
  x = ~TAXA_CESAREA,
  y = ~reorder(UF, TAXA_CESAREA),
  type = 'bar',
  orientation = 'h',
  marker = list(color = ~TAXA_CESAREA, colorscale = 'Blues'),
  text = ~paste0(round(TAXA_CESAREA, 1), "%"),
  textposition = 'outside',
  hoverinfo = 'text'
) %>%
  layout(
    title = "Taxa de Partos Cesáreos por UF em 2024",
    xaxis = list(title = "Taxa de Cesarianas (%)"),
    yaxis = list(title = "UF"),
    margin = list(l = 50, r = 50, t = 50, b = 50)
  )

print(p_interactive)